In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("./dataset/Iris.csv")

In [20]:
df

Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0      1            5.1           3.5            1.4           0.2   
1      2            4.9           3.0            1.4           0.2   
2      3            4.7           3.2            1.3           0.2   
3      4            4.6           3.1            1.5           0.2   
4      5            5.0           3.6            1.4           0.2   
..   ...            ...           ...            ...           ...   
145  146            6.7           3.0            5.2           2.3   
146  147            6.3           2.5            5.0           1.9   
147  148            6.5           3.0            5.2           2.0   
148  149            6.2           3.4            5.4           2.3   
149  150            5.9           3.0            5.1           1.8   

            Species  
0       Iris-setosa  
1       Iris-setosa  
2       Iris-setosa  
3       Iris-setosa  
4       Iris-setosa  
..              ...  
145  Iris-virginica  
146  Iris-virginica  
147  Iris-virginica  
148  Iris-virginica  
149  Iris-virginica  

[150 rows x 6 columns]

In [21]:
df.isna().sum()

Id               0
SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64

In [22]:
df.drop(columns="Id", inplace=True)

## Encoding

In [23]:
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
y_enc = one_hot_enc.fit_transform(df[["Species"]])

## Split data

In [24]:
# Drop original Y
x = df.drop(columns="Species")

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    x, 
    y_enc, 
    test_size = 0.2, 
    random_state = 42 #for reproducibility
)

In [26]:
## random forest (Baseline Model)
from sklearn.ensemble import RandomForestClassifier

#to poerform gridsearch for each model
from sklearn.model_selection import GridSearchCV

# Grid search setup
rf_grid = RandomForestClassifier()

gr_space = {
    'max_depth': [3,5,7,10,20,30],
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': [10, 20, 30 , 40],
    'min_samples_leaf': [1, 2, 4]
}

rf_model_grid = GridSearchCV(
    rf_grid,
    gr_space,
    cv = 3,
    scoring='f1_macro',
    verbose = 3,
    n_jobs=-1 #all cores
)

In [27]:
x_train.shape

(120, 4)

In [28]:
y_train.shape

(120, 3)

In [29]:
y_train

Species_Iris-setosa  Species_Iris-versicolor  Species_Iris-virginica
22                   1.0                      0.0                     0.0
15                   1.0                      0.0                     0.0
65                   0.0                      1.0                     0.0
11                   1.0                      0.0                     0.0
42                   1.0                      0.0                     0.0
..                   ...                      ...                     ...
71                   0.0                      1.0                     0.0
106                  0.0                      0.0                     1.0
14                   1.0                      0.0                     0.0
92                   0.0                      1.0                     0.0
102                  0.0                      0.0                     1.0

[120 rows x 3 columns]

In [30]:
# Fitting
rf_model_result = rf_model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[CV 2/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=100;, score=0.895 total time=   0.1s
[CV 1/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=100;, score=0.968 total time=   0.1s
[CV 3/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=100;, score=0.979 total time=   0.1s
[CV 1/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=200;, score=0.968 total time=   0.2s
[CV 2/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=200;, score=0.895 total time=   0.2s
[CV 3/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=200;, score=0.979 total time=   0.2s
[CV 2/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=300;, score=0.895 total time=   0.3s
[CV 1/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=300;, score=0.968 total time=   0.3s
[CV 1/3] END max_depth=3, max_features=10, min_samples_leaf=1, n_estimators=400;, score=0.968 to

In [32]:
rf_model_result

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10, 20, 30],
                         'max_features': [10, 20, 30, 40],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='f1_macro', verbose=3)

In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer